## An agent to validate radiology report based on ACR guidelines
The American College of Radiology (ACR), as the leading professional organization for radiologists, has established comprehensive guidelines for standardized radiology reporting to ensure diagnostic accuracy and patient care quality. In many healthcare settings, the current workflow requires senior radiologists to review and validate reports generated by junior colleagues—a critical but time-intensive process that creates bottlenecks in medical imaging departments already struggling with high workloads. To streamline this process while maintaining quality standards, Amazon NOVA's advanced multimodal AI capabilities can be leveraged to automatically evaluate radiology reports against ACR's established criteria, providing real-time feedback and guidance. This AI-assisted solution can instantly identify missing elements, suggest improvements based on ACR guidelines, and help junior radiologists generate more accurate and complete reports, ultimately reducing the review burden on senior radiologists while maintaining high reporting standards.

### Prepare the data
The ACR validation documents are in the folder ACRdocs. Upload the same in an S3 bucket. Change the name of the S3 bucket in the lambda function on line 38 in `lambda_handler`

In [ ]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

### Configure `AWS_PROFILE`
Configure the `AWS_PROFILE` using the usual process as described [here](https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-files.html).

If you are using an EC2 instance for your programming. You can assign EC2 instance a ROLE which has the appropriate access as mentioned in the README file. In such a case, you do not need to configure the `AWS_PROFILE`.

In [ ]:
import os
os.environ['AWS_PROFILE'] = 'rad-agent'

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

#### Configure the `boto3` clients

In [ ]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda', region_name='us-west-2')
bedrock_agent_client = boto3.client('bedrock-agent', region_name='us-west-2')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')

In [ ]:
session = boto3.session.Session(region_name='us-west-2')
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

### Write the agent instruction

In [ ]:
# configuration variables
suffix = f"{region}-{account_id}"
stack_name = "RadiologyReportValidator"
agent_name = f"{stack_name}-Agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}-{stack_name}"
agent_role_name = f'AmazonBRkExRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for Validating Radiology Reports"
agent_instruction = "You are a Radiology Report Validator, helping junior radiologist \
                    write reports in adherence to the ACR guidance criterion. Does the radiology report adheres to the ACR guidelines mentioned in the document? \
                    Is it detailed enough to provide a diagnosis? \
                    Is the report missing any key anatomical structures? \
                    Does the report meet the \
                    quality standards of the ACR guidelines? Please provide a terse actionable feedback and do not try to summarize the report itself. ?"
agent_action_group_name = "RadiologyActionGroup"
agent_action_group_description = "Actions for Validating Radiology Reports or from Machine Learning Models"
agent_alias_name = f"{agent_name}-alias"


### Assign the roles

In [ ]:
lambda_function_role = f"{stack_name}-LambdaExecutionRole"
lambda_function_name = f"{stack_name}-{account_id}-Lambda"
print(lambda_function_name)

# attach the s3 full access policy to the Lambda function role
s3_full_access_policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
dynamodb_full_access_policy_arn = "arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess"
bedrock_policy_arn = "arn:aws:iam::aws:policy/AmazonBedrockFullAccess"
AWSLambdaBasicExecutionRole_arn = "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"

# create role for the Lambda function
try:
    iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }),
        Description="Role for Lambda function to access S3 and other resources"
    )

    iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=s3_full_access_policy_arn
        
    )
    iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=dynamodb_full_access_policy_arn
    )

except Exception as e:
    print("Error creating role: ", e)
    print("Role already exists. Continuing...")
    



In [ ]:
Role=iam_client.get_role(RoleName=lambda_function_role)['Role']['Arn']
print(lambda_function_role)

In [ ]:
iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=AWSLambdaBasicExecutionRole_arn
    )
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn=bedrock_policy_arn
)
# assign the created role to a lambda function
lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Role=iam_client.get_role(RoleName=lambda_function_role)['Role']['Arn'],
    Timeout=900
)

#### Write and Upload the lambda function

In [ ]:
# Upload the lambda function
# Package up the lambda function code and deploy to Lambda function
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda/lambda_function.py", arcname="lambda_function.py")
z.close()
zip_content = s.getvalue()

lambda_function = lambda_client.update_function_code(
    FunctionName=lambda_function_name,
    ZipFile=zip_content,
    )
lambda_function_arn = lambda_function['FunctionArn']


### Create IAM policies for agent


In [ ]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
try:
    agent_bedrock_policy = iam_client.create_policy(
        PolicyName=agent_bedrock_allow_policy_name,
        PolicyDocument=bedrock_policy_json
    )
except Exception:
    policy_arn = f"arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}"
    agent_bedrock_policy = iam_client.get_policy(
        PolicyArn=policy_arn)
    print(f"Policy {agent_bedrock_allow_policy_name} already exists")
    

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
try:
    agent_role = iam_client.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )


    # Pause to make sure role is created
    time.sleep(10)
        
except Exception as e:
    agent_role = iam_client.get_role(RoleName=agent_role_name)
    print(f"Error creating role: {e}. Make sure it doesnt exist already")

iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
print(agent_role_name)

### Create Agent

Only run it the first time you are running this code. After the first run, grab the agent id and uncomment the first line of the following cell. 
For grabbing the agent-id, from the AWS Console page. Go to Bedrock---> Builder Tools ---> Agents to find your agent. Click on the agent to find the ID of the agent.

In [ ]:
agent_id = 'JEIWSMP9Y1'
try:
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
    agent_id = response['agent']['agentId']
    print(response)

except:
    print("Agent already exists, skipping creation")
    # agent_info = [agent for agent in bedrock_agent_client.list_agents()['agentSummaries'] if agent['agentName']==agent_name][0]
    # print(agent_info)
    # agent_id = agent_info['agentId']
    agent_version = bedrock_agent_client.list_agent_versions(agentId=agent_id)['agentVersionSummaries'][0]['agentVersion']
    response = bedrock_agent_client.list_agent_action_groups(
        agentId=agent_id,
        agentVersion=agent_version
    )
    print(response['actionGroupSummaries'][0])
    action_group_id = response['actionGroupSummaries'][0]['actionGroupId']
    print(f"Using agent_id {agent_id} and action_group_id {action_group_id}")

## Write Agent functions
Here we create agent functions which we would like to call as the agent processes the input radiology report. A typical agent_function contains the following fields:
- `name`: The name of the function
- `description`: Description of the function
- `parameters`: A dictionary of parameters to be passed in the function 
The `parameter` field contains the name of the parameter, its description, type as well as whether it is a required variable 

In [ ]:
agent_functions = [
    {
        'name': 'run_validator',
        'description': 'validates the radiology report for a given report',
        'parameters': {
            "report": {
                "description": "The radiology report for a given patient",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'get_radiology_report',
        'description': 'Get the radiology report for a given patient from a dynamodb table. If the patient id is not found, \
            return an error message and ask the user to provide the patient id.',
        'parameters': {
            "patient_id": {
                "description": "The patient ID for a given patient",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'download_guidance_document',
        'description': 'Download the ACR guidance document from the S3 bucket. If the document is not \
        available, ask the user to provide the document.',
        'parameters': {
            'report': {
                'description': 'Name of the modality or the anatomical structure to download the ACR guidance document',
                'required': True,
                'type': 'string'
            }
        }
    },
    {
        'name': 'identify_anatomical_structures',
        'description': 'Identify which of the following terms is most related to the report. Here is a list of anatomical structures. Choose the closest \
            anatomical structure from the following list: \
                - Brain \
                - Breast \
                - Mammography \
                - Spine \
                - Chest \
                - Abdomen \
                - Pelvis \
                - Extremities\
                - Transthoracic \
                - Echocardiography\
                and return the value from the list. Find the closest match to the items mentioned in the list.',
        'parameters': {
            'report': {
                'description': 'The radiology report for a given patient',
                'required': True,
                'type': 'string'
            }
        }
    },
    
]

In [ ]:
try:
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)
except:
    print("Action group already exists")
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        actionGroupId=action_group_id,
        agentVersion='DRAFT',
        actionGroupExecutor={
            'lambda': lambda_function_arn
        },
        actionGroupName=agent_action_group_name,
        functionSchema={
            'functions': agent_functions
        },
        description=agent_action_group_description
    )
    print(agent_action_group_response)

### Permissions for the lambda function
In this lamda function, required permissions are given. Here we are using bedrock in the agent, so appropriate permissions are given.

In [ ]:
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId='allow_bedrock',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except: 
    print("Permission already exists")


#### Prepare the agent

In [ ]:
# if the agent is already created, prepare the agent using the agent_id 
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id,
)
print(response)

### A function to print agent responses

In [ ]:
def print_agent_interaction(response):
    """
    This function prints the agent interaction in a readable format.

    Args:
        response (dict): The response from the agent.
    """
    answer = ""    
    for event in response["completion"]:
        print(event.keys())
        print(event)
        if 'chunk' in event:
            chunk_obj = event['chunk']
            if 'bytes' in chunk_obj:
                # Decode the bytes and append to the answer
                chunk_data = chunk_obj['bytes'].decode('utf-8')
                answer += chunk_data
    return answer

## Test case

In [ ]:
session_id:str = str(uuid.uuid1())
test_query="Can you validate the radiology report for patient id 1236?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer = print_agent_interaction(response)

### Sample test on Breast Imaging

In [ ]:
test_query="Can you validate the radiology report for patient id 1236?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer = print_agent_interaction(response)


### Sample test on Chest Imaging

In [ ]:
test_query="Can you validate the radiology report for patient id 1489?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer = print_agent_interaction(response)

In [ ]:
test_query="Can you validate the radiology report for patient id 1389?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer = print_agent_interaction(response)

### Sample test on Transthoracic imaging

In [ ]:
test_query="Can you validate the radiology report for patient id 1179?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer = print_agent_interaction(response)

### Sample test on Abdominal Ultrasound

In [ ]:
test_query="Can you validate the radiology report for patient id 1143?"

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer = print_agent_interaction(response)